In [1]:
import datetime
import numpy as np
import pandas as pd
import requests
import scipy
import scipy.sparse as sp
import json

import os
import pickle
import json

from bs4 import BeautifulSoup

from utils import weatherapi
from utils import isw_preprocessing
from utils import text_preprocessing

from sklearn import preprocessing


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dusty3ntity/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dusty3ntity/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dusty3ntity/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
MODELS_FOLDER = "data/models"
OUTPUT_FOLDER = "data/prediction"

if not os.path.exists(OUTPUT_FOLDER):
   os.makedirs(OUTPUT_FOLDER)

ISW_DATA_FILE = "data/isw/reports_processed_v2.csv"
WEATHER_EVENTS_DATA_FILE = "data/alarms/merged_events.csv"

tfidf_transformer_model = "tfidf_transformer"
tfidf_transformer_version = "v1"

count_vectorizer_model = "count_vectorizer"
count_vectorizer_version = "v1"

decision_tree_model = "DTC"
logistic_regression_model = "LR"
multi_layer_perceptron_model = "MLP"
random_forest_classifier_model = "RFC"
sgd_classifier_model = "SGD"

## 1. Load models & data

In [21]:
tfidf = pickle.load(open(f"data/isw/{tfidf_transformer_model}_{tfidf_transformer_version}.pkl", "rb"))
cv = pickle.load(open(f"data/isw/{count_vectorizer_model}_{count_vectorizer_version}.pkl", "rb"))

label_encoder = pickle.load(open(f"data/isw/hour_conditions_label_encoder.pkl", "rb"))

DTC = pickle.load(open(f"{MODELS_FOLDER}/{decision_tree_model}.pkl", "rb"))
LR = pickle.load(open(f"{MODELS_FOLDER}/{logistic_regression_model}.pkl", "rb"))
MLP = pickle.load(open(f"{MODELS_FOLDER}/{multi_layer_perceptron_model}.pkl", "rb"))
RFC = pickle.load(open(f"{MODELS_FOLDER}/{random_forest_classifier_model}.pkl", "rb"))
SGD = pickle.load(open(f"{MODELS_FOLDER}/{sgd_classifier_model}.pkl", "rb"))

In [5]:
REGIONS_DICTIONARY_FILE = "data/data_before_lab_3/regions.csv"
df_regions = pd.read_csv(REGIONS_DICTIONARY_FILE)
df_regions.head(25)

,region,center_city_ua,center_city_en,region_alt,region_id
0,АР Крим,Сімферополь,Simferopol,Крим,1
1,Вінницька,Вінниця,Vinnytsia,Вінниччина,2
2,Волинська,Луцьк,Lutsk,Волинь,3
3,Дніпропетровська,Дніпро,Dnipro,Дніпропетровщина,4
4,Донецька,Донецьк,Donetsk,Донеччина,5
5,Житомирська,Житомир,Zhytomyr,Житомирщина,6
6,Закарпатська,Ужгород,Uzhgorod,Закарпаття,7
7,Запорізька,Запоріжжя,Zaporozhye,Запоріжжя,8
8,Івано-Франківська,Івано-Франківськ,Ivano-Frankivsk,Івано-Франківщина,9
9,Київська,Київ,Kyiv,Київщина,10


## 2. Setup prediction region

In [6]:
region = "Львівщина"
date = "2023-04-08"

city = df_regions[df_regions["region_alt"]==region]["center_city_en"].values[0]

## 3. Get weather

In [7]:
level = "hours"
location = f"{city},Ukraine"

file_name = f"weather___{city.lower()}__{date}.json"

In [8]:
if not os.path.isfile(f"{OUTPUT_FOLDER}/{file_name}"):
	city_weather_json = weatherapi.get_weather_json(location, date)
	json_object = json.dumps(city_weather_json, indent=4)

	with open(f"{OUTPUT_FOLDER}/{file_name}", "w") as outfile:
		outfile.write(json_object)
else:
	print(f"Weather data from the \nregion {region}; \ndate {date}; \n is ready")

Weather data from the 
region Львівщина; 
date 2023-04-08; 
 is ready


In [9]:
# weather_file_path = f"{OUTPUT_FOLDER}/{file_name}"
# weather_for_day_hourly = weather.get_weather_hourly_for_region(weather_file_path)
# weather_df = pd.DataFrame.from_dict(weather_for_day_hourly)

# Getting random weather from the dataset as the API is not ready yet
all_weather_by_hour = pd.read_csv("data/data_before_lab_3/all_weather_by_hour.csv")
weather_df = all_weather_by_hour.loc[all_weather_by_hour['day_datetime'] == "2022-04-08"]

weather_df["day_datetime"] = pd.to_datetime(weather_df["day_datetime"])
weather_df["city"] = weather_df["city_resolvedAddress"].apply(lambda x: x.split(",")[0])
weather_df["city"] = weather_df["city"].replace("Хмельницька область", "Хмельницький")

/var/folders/7p/g27ft7gd5hd6j8355sgsl7080000gn/T/ipykernel_15030/536417939.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_df["day_datetime"] = pd.to_datetime(weather_df["day_datetime"])
/var/folders/7p/g27ft7gd5hd6j8355sgsl7080000gn/T/ipykernel_15030/536417939.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_df["city"] = weather_df["city_resolvedAddress"].apply(lambda x: x.split(",")[0])
/var/folders/7p/g27ft7gd5hd6j8355sgsl7080000gn/T/ipykernel_15030/536417939.py:11: SettingWithCop

In [10]:
print(weather_df.shape)
weather_df.head(5)

(576, 68)


,city_latitude,city_longitude,city_resolvedAddress,city_address,city_timezone,city_tzoffset,day_datetime,day_datetimeEpoch,day_tempmax,day_tempmin,day_temp,day_feelslikemax,day_feelslikemin,day_feelslike,day_dew,day_humidity,day_precip,day_precipprob,day_precipcover,day_snow,day_snowdepth,day_windgust,day_windspeed,day_winddir,day_pressure,day_cloudcover,day_visibility,day_solarradiation,day_solarenergy,day_uvindex,day_severerisk,day_sunrise,day_sunriseEpoch,day_sunset,day_sunsetEpoch,day_moonphase,day_conditions,day_description,day_icon,day_source,day_preciptype,day_stations,hour_datetime,hour_datetimeEpoch,hour_temp,hour_feelslike,hour_humidity,hour_dew,hour_precip,hour_precipprob,hour_snow,hour_snowdepth,hour_preciptype,hour_windgust,hour_windspeed,hour_winddir,hour_pressure,hour_visibility,hour_cloudcover,hour_solarradiation,hour_solarenergy,hour_uvindex,hour_severerisk,hour_conditions,hour_icon,hour_source,hour_stations,city
1031,49.4407,32.0637,"Черкаси, Україна","Cherkasy,Ukraine",Europe/Kiev,2.0,2022-04-08,1649365200,16.8,5.3,10.1,16.8,3.6,9.4,3.1,63.5,2.4,100.0,12.5,0.0,0.0,44.3,25.6,225.4,999.9,73.3,23.4,158.1,13.6,6.0,10.0,06:14:37,1649387677,19:33:39,1649435619,0.22,"Rain, Partially cloudy",Partly cloudy throughout the day with late aft...,rain,obs,rain,remote,00:00:00,1649365200,9.4,7.9,53.78,0.5,0.0,0.0,0.0,0.0,NaN,21.6,9.7,157.8,1002.0,24.1,0.0,0.0,NaN,0.0,10.0,Clear,clear-night,obs,remote,Черкаси
1032,49.4407,32.0637,"Черкаси, Україна","Cherkasy,Ukraine",Europe/Kiev,2.0,2022-04-08,1649365200,16.8,5.3,10.1,16.8,3.6,9.4,3.1,63.5,2.4,100.0,12.5,0.0,0.0,44.3,25.6,225.4,999.9,73.3,23.4,158.1,13.6,6.0,10.0,06:14:37,1649387677,19:33:39,1649435619,0.22,"Rain, Partially cloudy",Partly cloudy throughout the day with late aft...,rain,obs,rain,remote,01:00:00,1649368800,8.5,7.6,56.32,0.3,0.0,0.0,0.0,0.0,NaN,16.9,6.5,162.9,1002.0,24.1,11.5,0.0,NaN,0.0,10.0,Clear,clear-night,obs,remote,Черкаси
1033,49.4407,32.0637,"Черкаси, Україна","Cherkasy,Ukraine",Europe/Kiev,2.0,2022-04-08,1649365200,16.8,5.3,10.1,16.8,3.6,9.4,3.1,63.5,2.4,100.0,12.5,0.0,0.0,44.3,25.6,225.4,999.9,73.3,23.4,158.1,13.6,6.0,10.0,06:14:37,1649387677,19:33:39,1649435619,0.22,"Rain, Partially cloudy",Partly cloudy throughout the day with late aft...,rain,obs,rain,remote,02:00:00,1649372400,7.8,7.8,59.49,0.4,0.0,0.0,0.0,0.0,NaN,11.9,4.0,126.6,1001.0,24.1,47.6,0.0,NaN,0.0,10.0,Partially cloudy,partly-cloudy-night,obs,remote,Черкаси
1034,49.4407,32.0637,"Черкаси, Україна","Cherkasy,Ukraine",Europe/Kiev,2.0,2022-04-08,1649365200,16.8,5.3,10.1,16.8,3.6,9.4,3.1,63.5,2.4,100.0,12.5,0.0,0.0,44.3,25.6,225.4,999.9,73.3,23.4,158.1,13.6,6.0,10.0,06:14:37,1649387677,19:33:39,1649435619,0.22,"Rain, Partially cloudy",Partly cloudy throughout the day with late aft...,rain,obs,rain,remote,03:00:00,1649376000,7.2,7.2,63.34,0.7,0.0,0.0,0.0,0.0,NaN,8.3,4.3,111.8,1001.0,24.1,86.3,0.0,NaN,0.0,10.0,Partially cloudy,partly-cloudy-night,obs,remote,Черкаси
1035,49.4407,32.0637,"Черкаси, Україна","Cherkasy,Ukraine",Europe/Kiev,2.0,2022-04-08,1649365200,16.8,5.3,10.1,16.8,3.6,9.4,3.1,63.5,2.4,100.0,12.5,0.0,0.0,44.3,25.6,225.4,999.9,73.3,23.4,158.1,13.6,6.0,10.0,06:14:37,1649387677,19:33:39,1649435619,0.22,"Rain, Partially cloudy",Partly cloudy throughout the day with late aft...,rain,obs,rain,remote,04:00:00,1649379600,6.3,5.5,70.35,1.3,0.0,0.0,0.0,0.0,NaN,9.4,5.0,97.7,1001.0,24.1,64.2,0.0,NaN,0.0,10.0,Partially cloudy,partly-cloudy-night,obs,remote,Черкаси


In [11]:
weather_df_v2 = pd.merge(weather_df, df_regions, left_on="city", right_on="center_city_ua")

## 3. Get ISW report

In [12]:
date = "2023-04-08"

BASE_URL = "https://www.understandingwar.org/backgrounder/russian-offensive-campaign-assessment"
months = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]
date_arr = date.split("-")

url = f"{BASE_URL}-{months[int(date_arr[1])-1]}-{int(date_arr[2])}-{int(date_arr[0])}"
print(url)

page = requests.get(url)
isw_report_html_file = f"{OUTPUT_FOLDER}/isw___{date}.html"
with open(isw_report_html_file, "wb+") as f:
  f.write(page.content)

https://www.understandingwar.org/backgrounder/russian-offensive-campaign-assessment-april-8-2023


In [13]:
with open(isw_report_html_file, "r", encoding="utf8") as cfile:
		html = BeautifulSoup(cfile.read(), features="html.parser")
		content_html = str(html.body.find("div", attrs={"class": "field-type-text-with-summary"}))

		content_html = isw_preprocessing.preprocess_page_html(content_html)
		content_text_lemm = text_preprocessing.text_preprocess(content_html, "lemm")

In [24]:
word_count_vector = cv.transform([content_text_lemm])

In [25]:
word_count_vector

<1x6524 sparse matrix of type '<class 'numpy.int64'>'
	with 945 stored elements in Compressed Sparse Row format>

In [26]:
tfidf_vector = tfidf.transform(word_count_vector)

In [27]:
tfidf_vector

<1x6524 sparse matrix of type '<class 'numpy.float64'>'
	with 945 stored elements in Compressed Sparse Row format>

## 4. Data preparation

In [28]:
weather_exclude = [
"day_feelslikemax",
"day_feelslikemin",
"day_sunriseEpoch",
"day_sunsetEpoch",
"day_description",
"city_latitude",
"city_longitude",
"city_address",
"city_timezone",
"city_tzoffset",
"day_feelslike",
"day_precipprob",
"day_snow",
"day_snowdepth",
"day_windgust",
"day_windspeed",
"day_winddir",
"day_pressure",
"day_cloudcover",
"day_visibility",
"day_severerisk",
"day_conditions",
"day_icon",
"day_source",
"day_preciptype",
"day_stations",
"hour_icon",
"hour_source",
"hour_stations",
"hour_feelslike"
]

fields_to_exclude = [
	"city_resolvedAddress",
	"day_datetime",
	"day_datetimeEpoch",
	"hour_datetime",
	"hour_datetimeEpoch",
	"city",
	"region",
	"center_city_ua",
	"center_city_en"
]

tmp_fields_to_exclude = [
	"day_sunrise",
	"day_sunset",
	"hour_preciptype",
	"hour_conditions",
	"hour_solarenergy",
	"region_alt"
]

df_work_v2 = weather_df_v2.drop(weather_exclude, axis=1)
df_work_v2 = df_work_v2.drop(fields_to_exclude, axis=1)

In [29]:
df_work_v2["hour_conditions"] = df_work_v2["hour_conditions"].apply(lambda x: x.split(",")[0])
df_work_v2["hour_conditions_id"] = label_encoder.transform(df_work_v2["hour_conditions"])

In [30]:
df_work_v3 = df_work_v2.drop(tmp_fields_to_exclude, axis=1)

In [31]:
df_work_v3.head(10)

,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,day_solarradiation,day_solarenergy,day_uvindex,day_moonphase,hour_temp,hour_humidity,hour_dew,hour_precip,hour_precipprob,hour_snow,hour_snowdepth,hour_windgust,hour_windspeed,hour_winddir,hour_pressure,hour_visibility,hour_cloudcover,hour_solarradiation,hour_uvindex,hour_severerisk,region_id,hour_conditions_id
0,16.8,5.3,10.1,3.1,63.5,2.4,12.5,158.1,13.6,6.0,0.22,9.4,53.78,0.5,0.0,0.0,0.0,0.0,21.6,9.7,157.8,1002.0,24.1,0.0,0.0,0.0,10.0,23,0
1,16.8,5.3,10.1,3.1,63.5,2.4,12.5,158.1,13.6,6.0,0.22,8.5,56.32,0.3,0.0,0.0,0.0,0.0,16.9,6.5,162.9,1002.0,24.1,11.5,0.0,0.0,10.0,23,0
2,16.8,5.3,10.1,3.1,63.5,2.4,12.5,158.1,13.6,6.0,0.22,7.8,59.49,0.4,0.0,0.0,0.0,0.0,11.9,4.0,126.6,1001.0,24.1,47.6,0.0,0.0,10.0,23,4
3,16.8,5.3,10.1,3.1,63.5,2.4,12.5,158.1,13.6,6.0,0.22,7.2,63.34,0.7,0.0,0.0,0.0,0.0,8.3,4.3,111.8,1001.0,24.1,86.3,0.0,0.0,10.0,23,4
4,16.8,5.3,10.1,3.1,63.5,2.4,12.5,158.1,13.6,6.0,0.22,6.3,70.35,1.3,0.0,0.0,0.0,0.0,9.4,5.0,97.7,1001.0,24.1,64.2,0.0,0.0,10.0,23,4
5,16.8,5.3,10.1,3.1,63.5,2.4,12.5,158.1,13.6,6.0,0.22,5.7,74.39,1.5,0.0,0.0,0.0,0.0,11.5,6.5,104.9,1000.0,24.1,38.3,0.0,0.0,10.0,23,4
6,16.8,5.3,10.1,3.1,63.5,2.4,12.5,158.1,13.6,6.0,0.22,5.3,76.48,1.5,0.0,0.0,0.0,0.0,13.0,7.6,107.8,999.0,24.1,100.0,0.0,0.0,10.0,23,3
7,16.8,5.3,10.1,3.1,63.5,2.4,12.5,158.1,13.6,6.0,0.22,5.4,75.95,1.5,0.0,0.0,0.0,0.0,13.3,7.6,120.6,999.0,24.1,82.3,14.0,0.0,10.0,23,4
8,16.8,5.3,10.1,3.1,63.5,2.4,12.5,158.1,13.6,6.0,0.22,6.8,71.46,2.0,0.0,0.0,0.0,0.0,13.3,7.9,124.2,999.0,24.1,100.0,90.0,1.0,10.0,23,3
9,16.8,5.3,10.1,3.1,63.5,2.4,12.5,158.1,13.6,6.0,0.22,9.0,64.21,2.6,0.0,0.0,0.0,0.0,17.6,7.2,140.7,998.0,24.1,91.4,176.0,2.0,10.0,23,3


In [37]:
tfidf_matrix = tfidf_vector

for i in range(0, 575):
	tfidf_matrix = sp.vstack((tfidf_matrix, tfidf_vector), format="csr")

In [38]:
tfidf_vector

<1x6524 sparse matrix of type '<class 'numpy.float64'>'
	with 945 stored elements in Compressed Sparse Row format>

In [39]:
tfidf_matrix

<29x6524 sparse matrix of type '<class 'numpy.float64'>'
	with 27405 stored elements in Compressed Sparse Row format>

In [40]:
df_work_v4_csr = sp.csr_matrix(df_work_v3.values)
df_work_v4_csr

<576x29 sparse matrix of type '<class 'numpy.float64'>'
	with 13725 stored elements in Compressed Sparse Row format>

In [41]:
df_all_features = sp.hstack((df_work_v4_csr, tfidf_matrix), format="csr")

ValueError: Mismatching dimensions along axis 0: {576, 29}

In [ ]:
hours_alarm_schedule = DTC.predict(df_all_features)

In [ ]:
hours_alarm_schedule